In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import glob
import tqdm

from ukbb_recessive.data_collection.dataset import RegressionDataset

In [ ]:
datasets_save_path = '.../450k/datasets/sampling/synonymous'

#_____________________________________________________________________________________________________________________________
s_het_paths = {
    "Roulette": ".../450k/selection_roulette/s_het_roulette_gencode-v34.csv",
}

#_____________________________________________________________________________________________________________________________
# phenotypic paths
age_children_path = ".../450k/phenotypes/age_children_data_participant.tsv" 
pca_path = ".../450k/phenotypes/PC_participant.tsv"
other_features_path = (
    ".../450k/phenotypes/phenotypes_of_interest_participant.tsv"
)

#_____________________________________________________________________________________________________________________________
# samples of interest: European & non-related
european_non_rel_samples = (
    ".../450k/samples/european_non_related_no_withdrawal_to_include_450k.txt"
)

#_____________________________________________________________________________________________________________________________
# plps paths

variants_paths_cfg = {
    'synonymous' : {
        'cohort_files': glob.glob(".../450k/RAP_output_per_chr/filtered_plps/synonymous_rare/chr*"), 
        'all_variants_folder': ".../450k/plp_selection/synonymous_rare/all_chr_total_presumable_plps_HFE_final_sorted_samples"
    },
}
#_____________________________________________________________________________________________________________________________
# gene info
gencode = ".../genCode/GRCh38/v34/gencode.v34.GRCh38.txt"

#gene panel
gene_panel = pd.read_csv(".../450k/regions/gene-panel-gencode-v34.txt", header=None)
gene_panel.columns = ['Gene name', 'Gene panel']

gene_panel.loc[~gene_panel['Gene panel'].str.contains('ID-total'), 'Gene panel'] = 'AR_without_ID'
gene_panel['Gene panel'].value_counts()

# Collect phenotypic info

In [ ]:
with open(european_non_rel_samples, 'r') as f:
    european_non_rel_samples = [l.strip() for l in f.readlines()]

print (f"Number of european non-related samples: {len(european_non_rel_samples)}")

features = RegressionDataset(
    age_children_path = age_children_path, 
    pca_path = pca_path, 
    other_features_path = other_features_path,
    samples_list = european_non_rel_samples, 
).collect_phenotypic_features()

In [ ]:
print ("Mistakes children:", sum((~features['number_of_children_fathered'].isnull()) & (~features['number_of_live_births'].isnull())))
print ("Missing children:", sum((features['number_of_children_fathered'].isnull()) & (features['number_of_live_births'].isnull())))
print ("Missing children:", sum(features['number_of_children_MF'].isnull()))
print ("Childlessness:", int(features['childlessness'].sum()))
print ("Childlessness missing:", sum(features['childlessness'].isnull()))


features[['number_of_children_fathered', 'number_of_live_births', 'number_of_children_MF']].hist(bins=8, figsize=(20, 3), layout=(1, 3))

plt.show()

In [ ]:
print ("Total number of individuals:", features.shape[0])
print ("Number of individuals with university degree:", int(features['uni_1_0_including_none'].sum()))
print ("Number of individuals with higher educational degree:", int(features['higher_education_including_none'].sum()))

qualifications = features[['eid', 'qualifications']].copy()
qualifications['qualifications'] = qualifications['qualifications'].apply(lambda x: x.split('|'))
qualifications = qualifications.explode('qualifications')
qualifications.drop_duplicates().groupby('qualifications').count().plot.barh()

plt.show()

In [ ]:
print ("Number of individuals with partner:", int(features['living_with_a_partner'].sum()))
print()
print ("Number of individuals with mental health ICD diagnosis:", int(features['ICD_mental_health_yes_no'].sum()))
print ("Number of individuals with mental health questionnarie:", int(features['mental_health_Q'].sum()))
print()
print ("Number of individuals with email:", int(features['email'].sum()))
print()
print ("Number of ever had sex:", int(features['ever_had_sex'].sum()))
print ("Number of never had sex:", int((1 - features['ever_had_sex']).sum()))
print()
print ("Number has gp record sex:", int(features['has_gp_record'].sum()))
print()
print ("Number left-handed:", int(features['is_left_handed'].sum()))
print()
print ("Number blond:", int(features['is_blond'].sum()))
print()
print ("Number infertility:", int(features['ICD_infertility'].sum()))

In [ ]:
features.groupby('gender').count()[['eid']]

# Collect s-het info

## Prepare gene panels

In [ ]:
# Create gene panels recessive
recessive_panels_dict = gene_panel.groupby('Gene panel').agg({'Gene name': lambda x: list(x)}).to_dict()['Gene name']
recessive_panels_dict['all'] = None

s_het_cfg = {
    'synonymous' : recessive_panels_dict
}

for dataset_type in s_het_cfg:
    print (f"Dataset = {dataset_type}")
    for panel, gene_list in s_het_cfg[dataset_type].items():

        print (f"\tPanel={panel}, number of genes={len(gene_list) if gene_list is not None else 'NA' }")
    print()

## Collect s_het data

In [ ]:
all_sampling_datasets = []

for i in range(1000):
    print (f"\n\nCollecting data for iteration = {i}\n")
    result = {}

    for s_het_key, s_het_path in s_het_paths.items():
        print (f"Collecting data for {s_het_key}\n")

        for dataset_type in s_het_cfg:
            print (f"Dataset = {dataset_type}")

            for panel, gene_list in s_het_cfg[dataset_type].items():

                print (f"\tPanel={panel}, number of genes={len(gene_list) if gene_list is not None else 'NA' }")
                print ("_____________________")
                s_het_features = RegressionDataset(
                    het_occurrence_threshold=20., 
                    all_plps_file=f"{variants_paths_cfg[dataset_type]['all_variants_folder']}/sample_{i}.csv", 
                    s_het_file=s_het_path,
                    cohort_plps_files=variants_paths_cfg[dataset_type]['cohort_files'], 
                    genes_list=gene_list,
                    dataset=dataset_type
                ).collect_variant_features()
                
                # rename columns
                for col in s_het_features.columns.tolist():
                    if col != 'eid':
                        s_het_features = s_het_features.rename(columns={col: f"{col}_{dataset_type}_{panel}"})

                if s_het_key not in result:
                    result[s_het_key] = s_het_features
                else:
                    result[s_het_key] = result[s_het_key].merge(s_het_features, on="eid", how='outer')
                print ("_____________________")
                
    all_sampling_datasets.append(result)

In [ ]:
len(all_sampling_datasets)

In [ ]:
# check shapes
result = all_sampling_datasets[0]
for s_het_key in s_het_paths:
    print (f"Shape for variants features of {s_het_key}:", result[s_het_key].shape)

# Merge and fill-in NA

In [ ]:
 # fill NA for s_het with PLP but no information about s_het
fill_na_shet = True

for idx, result in enumerate(all_sampling_datasets):

    for s_het_key in s_het_paths:
        print (f"Merging data for {s_het_key}")

        result[s_het_key] = features.merge(result[s_het_key], on='eid', how='left')

        for dataset_type in s_het_cfg:
            print (f"\tDataset = {dataset_type}")

            for panel, gene_list in s_het_cfg[dataset_type].items():
                
                columns = [f"{col}_{dataset_type}_{panel}" for col in ['s_het', 'mutations_cnt', 'has_mutation']]

                # fills NAs with 0s
                result[s_het_key][columns] = result[s_het_key][columns].fillna(0.)

                # fill NAs for s_het of recessive PLP carriers with no s_het information
                if fill_na_shet:
                    result[s_het_key].loc[(result[s_het_key][f'mutations_cnt_{dataset_type}_{panel}'] > 0) & 
                                        (result[s_het_key][f's_het_{dataset_type}_{panel}'] == 0),  f's_het_{dataset_type}_{panel}'] = None
                    
    all_sampling_datasets[idx] = result

In [ ]:
# check shapes
result = all_sampling_datasets[0]
for s_het_key in s_het_paths:
    print (f"Shape for all features of {s_het_key}:", result[s_het_key].shape)

In [ ]:
for idx, result in tqdm.tqdm(enumerate(all_sampling_datasets)):
    for s_het_key in s_het_paths:
        result[s_het_key].to_csv(f'{datasets_save_path}/{s_het_key}_sample_{idx + OFFSET}.csv', sep='\t', index=False)

In [ ]:
! ls -lah .../450k/datasets_synonymous/samples/Roulette_* | wc -l 